In [1]:
from datasets import load_metric, Dataset
import numpy as np
import pandas as pd
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
import torch


In [2]:
eval_processor = Wav2Vec2Processor.from_pretrained(f"./icfoss-model/")
eval_model = Wav2Vec2ForCTC.from_pretrained(f"./epoch2/") 
eval_model.to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elemen

In [3]:
test_data = Dataset.from_pandas(pd.read_pickle('./idicTTS.pkl')[['target_text','speech']])

In [ ]:
test_data['speech'][0]

In [4]:
wer = load_metric("wer")

C:\Users\nisha\AppData\Local\Temp\ipykernel_10164\364875503.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer = load_metric("wer")


In [5]:
test_data

Dataset({
    features: ['target_text', 'speech'],
    num_rows: 6895
})

In [6]:
def evaluate(batch):
    inputs = eval_processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = eval_model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = eval_processor.batch_decode(pred_ids)
    return batch


In [7]:
result = test_data.map(evaluate, batched=True, batch_size=8)

Map:   0%|          | 0/6895 [00:00<?, ? examples/s]

In [8]:
print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["target_text"])))

WER: 33.074747


In [9]:
result["pred_strings"]

['നിങ്ങളുടെ പിതാവിന്റെ ആഗ്രഹങ്ങൾ നിറവേറ്റാനാണ് നിങ്ങൾ ഇച്ഛിക്കുന്നത്',
 'എന്റെ കൈ്യിൽ അത്തരം പേരില്ലുള്ള ഒന്നും ഇല്ല',
 'വ്യത്യസ്തൈനം ഓർക്കെടുകളും ദേശീയോദ്യാനത്തിൽ കാണപ്പെടുന്നു',
 'മത്സ്യബന്ധനമാണ് പട്ടണത്തിലെ ആളുകളുടെ പ്രധാന തൊഴിൽ',
 'ബിയറിന്റെ സാമ്പളിൽ നിന്ന് മെത്തനോൾ കണ്ടെത്താൻ കഴിയില്ല',
 'ഒട്ടേറെ മുസ്ലീം ക്രിസ്റ്റ്യൻ സഹോദരന്മാർക്കും ഇതേ അഭിപ്രായമുണ്ട്',
 'അതിന്റെ വിസ്ഫോടന ഘോഷം കേട്ട് മേഖമൈലുകൾ ആനന്ത നൃത്തം തുടങ്ങണം',
 'വിവാഹശേഷം ചുരുങ്ങിയത് എട്ട് മാസമെങ്കിലും കഴിഞ്ഞാവാം ഗർഭദാരണം',
 'പക്ഷേ സന്യാസിയായ ശങ്കരാചാര്യർക്ക് വിഷയത്തിൽ ഞാനമില്ലായിരുന്നു',
 'ആശാരിയും മറ്റുള്ളവരും ചേർന്ന് മണ്ണെട്ട് കുഴിയിൽ കൊടിമരം ഉറപ്പിക്കുന്നു',
 'അമേരിക്കയുടെ സുരക്ഷിതത്തത്തിനു നേരെയുള്ള വെല്ലവിളി അവർ പുറം ലോകത്തെ അറിയിച്ചു',
 'അതുകൊണ്ട് തന്നെ അവിഡങ്ങളിലുള്ള മറ്റ് ജീവികൾക്ക് പൂച്ചകളോട് അധികം പേഡിയുണ്ടാകാറില്ല',
 'ശ്രീക്കോവിലിന്റെ ഭിത്തികൾ കരിങ്കല്ലും മണ്ണും ചേർത്ത് നിർമ്മിച്ചിരിക്കുന്നു',
 'അതിനാൽ ഓരോ സമയവും അൽപാൽപമായി ഹബിൾ താഴോട്ട് വന്നുകൊണ്ടിരിക്കുന്നു',
 'അശോകനുശേഷം മഗതയ്ക്ക് ബലക്ഷയന്നേരിട്ടു',
 'പ്രകൃതി

In [10]:
result["target_text"]

['നിങ്ങളുടെ പിതാവിന്റെ ആഗ്രഹങ്ങൾ നിറവേറ്റാനാണ് നിങ്ങൾ ഇച്ഛിക്കുന്നത്',
 'എന്റെ കയ്യിൽ അത്തരം പേരിലുള്ള ഒന്നും ഇല്ല',
 'വ്യത്യസ്ത ഇനം ഓർക്കിഡുകളും ദേശീയോദ്യാനത്തിൽ കാണപ്പെടുന്നു',
 'മത്സ്യബന്ധനമാണ് പട്ടണത്തിലെ ആളുകളുടെ പ്രധാന തൊഴിൽ',
 'ബിയറിന്റെ സാമ്പിളിൽ നിന്ന് മെഥനൊൾ കണ്ടെത്താൻ കഴിയില്ല',
 'ഒട്ടേറെ മുസ്ലിം കൃസ്ത്യൻ സഹോദരന്മാർക്കും ഇതേ അഭിപ്രായമുണ്ട്',
 'അതിന്റെ വിസ്ഫോടനഘോഷം കേട്ട് മേഘമയിലുകൾ ആനന്ദനൃത്തം തുടങ്ങണം',
 'വിവാഹശേഷം ചുരുങ്ങിയത് എട്ട് മാസമെങ്കിലും കഴിഞ്ഞാവാം ഗർഭധാരണം',
 'പക്ഷേ സന്ന്യാസിയായ ശങ്കരാചാര്യർക്ക് വിഷയത്തിൽ ജ്ഞാനമില്ലായിരുന്നു',
 'ആശാരിയും മറ്റുള്ളവരും ചേർന്ന് മണ്ണിട്ട് കുഴിയിൽ കൊടിമരം ഉറപ്പിക്കുന്നു',
 'അമേരിക്കയുടെ സുരക്ഷിതത്വത്തിനു നേരെയുള്ള വെല്ലുവിളി അവർ പുറംലോകത്തെ അറിയിച്ചു',
 'അതുകൊണ്ട് തന്നെ അവിടങ്ങളിലുള്ള മറ്റ് ജീവികൾക്ക് പൂച്ചകളോട് അധികം പേടിയുണ്ടാകാറില്ല',
 'ശ്രീകോവിലിന്റെ ഭിത്തികൾ കരിങ്കല്ലും മണ്ണും ചേർത്ത് നിർമ്മിച്ചിരിക്കുന്നു',
 'അതിനാൽ ഓരോ സമയവും അല്പാല്പമായി ഹബിൾ താഴോട്ടു വന്നുകൊണ്ടിരിക്കുന്നു',
 'അശോകനുശേഷം മഗധയ്ക്ക് ബലക്ഷയം നേരിട്ടു',
 'പ്രകൃതിയുമ